In [24]:
import pandas as pd

# Read the 'prevalence' sheet using openpyxl
df = pd.read_excel(
    'full_data.xlsx',
    sheet_name='Prevalence',
    engine='openpyxl'
)

# Save to CSV
df.to_csv('prevalence.csv', index=False)

print("Saved 'prevalence' sheet to 'prevalence.csv'.")


Saved 'prevalence' sheet to 'prevalence.csv'.


In [25]:
import pandas as pd

# 1. Load the sheet (use openpyxl for .xlsx)
df = pd.read_csv('prevalence.csv')

# 2. Forward-fill PMID so every row has the correct value
df['PMID'] = df['PMID'].ffill()

# 3. Save to CSV
df.to_csv('prevalence.csv', index=False)

print("Done — all PMIDs filled and saved to 'prevalence_filled.csv'.")


Done — all PMIDs filled and saved to 'prevalence_filled.csv'.


In [26]:
import pandas as pd

# 1. Load and reset index
df = pd.read_csv('prevalence.csv').reset_index(drop=True)

# 2. Drop rows where any of these are NA: age_start, age_end, year_start, year_end, Sample size, proportion
df = df.dropna(subset=[
    'age_start', 'age_end', 
    'year_start', 'year_end',
    'Sample size', 'proportion'
])

# 3. Cast age and year columns to integers
for c in ['age_start','age_end','year_start','year_end']:
    df[c] = df[c].astype(int)

# 4. Build 'area' from Nation (if non-NA) otherwise Region
df['area'] = df['Nation'].fillna(df['Region'])

# 5. Set constant data_type = 'p'
df['data_type'] = 'p'

# 6. Lowercase sex
df['sex'] = df['Sex'].str.lower()

# 7. Reset index into its own column, then select & reorder
out = df.reset_index()[[
    'index',
    'sex',
    'area',
    'Sample size',
    'data_type',
    'proportion',
    'standard error',
    'age_start',
    'age_end',
    'year_start',
    'year_end'
]].rename(columns={
    'Sample size':    'effective_sample_size',
    'proportion':     'value',
    'standard error': 'standard_error'
})

# 8. Save final CSV
out.to_csv('prevalence.csv', index=False)

print("Saved with columns: index, sex, area, effective_sample_size, data_type, value, standard_error, age_start, age_end, year_start, year_end.")


Saved with columns: index, sex, area, effective_sample_size, data_type, value, standard_error, age_start, age_end, year_start, year_end.


In [27]:
import pandas as pd

# Load the CSV
df = pd.read_csv('prevalence.csv')

# # Re-create your 'area' logic
# df['area'] = df['Nation'].fillna(df['Region'])

# Print unique area values
print(df['area'].unique())


['US' 'Austrailia' 'Netherland' 'Finland' 'Barbados' 'Italy' 'Greece'
 'Japan' 'France' 'Iceland' 'India' 'China' '7 countries' 'Norway'
 'Estonia' 'Northern Ireland' 'Spain' 'Greenland' 'Brazil' 'Taiwan'
 'Singapore' 'Thailand' 'UK' 'South Korea' 'United Kingdom' 'Germany'
 'Kenya' 'Netherlands' 'Ireland' 'Algeria' 'Portugal' 'Slovakia' 'Russia'
 'Iran']


In [28]:
# area 문자열 → ISO3 코드 매핑표
area_to_iso = {
    'US':                 'USA',
    'Austrailia':         'AUS',
    'Netherland':         'NLD',
    'Netherlands':        'NLD',
    'Finland':            'FIN',
    'Barbados':           'BRB',
    'Italy':              'ITA',
    'Greece':             'GRC',
    'Japan':              'JPN',
    'France':             'FRA',
    'Iceland':            'ISL',
    'India':              'IND',
    'China':              'CHN',
    'Norway':             'NOR',
    'Estonia':            'EST',
    'Northern Ireland':   'GBR', 
    'Spain':              'ESP',
    'Greenland':          'GRL',
    'Brazil':             'BRA',
    'Taiwan':             'TWN',
    'Singapore':          'SGP',
    'Thailand':           'THA',
    'UK':                 'GBR',
    'United Kingdom':     'GBR',
    'South Korea':        'KOR',
    'Germany':            'DEU',
    'Kenya':              'KEN',
    'Ireland':            'IRL',
    'Algeria':            'DZA',
    'Portugal':           'PRT',
    'Slovakia':           'SVK',
    'Russia':             'RUS',
    'Iran':               'IRN'
}

# 적용 예시
import pandas as pd

df = pd.read_csv('prevalence.csv')
# 기존 area 컬럼 재생성 로직

df['area'] = df['area'].map(area_to_iso)

# 3. Save to CSV
df.to_csv('prevalence.csv', index=False)

print(df[['area']].drop_duplicates().reset_index(drop=True))


   area
0   USA
1   AUS
2   NLD
3   FIN
4   BRB
5   ITA
6   GRC
7   JPN
8   FRA
9   ISL
10  IND
11  CHN
12  NaN
13  NOR
14  EST
15  GBR
16  ESP
17  GRL
18  BRA
19  TWN
20  SGP
21  THA
22  KOR
23  DEU
24  KEN
25  IRL
26  DZA
27  PRT
28  SVK
29  RUS
30  IRN


In [9]:
import pandas as pd
import numpy as np

# 1. Load the CSV
df = pd.read_csv('input_data.csv')

# 2. Find rows where value == 0 and fix
mask_zero = df['value'] == 0
df.loc[mask_zero, 'value'] = 1 / (2 * df.loc[mask_zero, 'effective_sample_size'])
df.loc[mask_zero, 'standard_error'] = np.sqrt(
    df.loc[mask_zero, 'value'] * (1 - df.loc[mask_zero, 'value']) / df.loc[mask_zero, 'effective_sample_size']
)

# 3. Cast effective_sample_size to integer
df['effective_sample_size'] = df['effective_sample_size'].astype(int)

# 4. Drop rows where value > 1
mask_gt1 = df['value'] > 1
num_gt1 = mask_gt1.sum()
df = df[~mask_gt1]

# 5. Report counts
print(f"Adjusted {mask_zero.sum()} rows where 'value' was zero.")
print(f"Removed {num_gt1} rows where 'value' exceeded 1.")

# 6. Save back to CSV
df.to_csv('input_data.csv', index=False)


Adjusted 0 rows where 'value' was zero.
Removed 2 rows where 'value' exceeded 1.


In [11]:
import pandas as pd
import numpy as np

# 1. Load the CSV
df = pd.read_csv('input_data.csv')

# 2. Drop rows where 'area' is missing or empty
mask_noarea = df['area'].isna() | (df['area'].astype(str).str.strip() == '')
num_noarea = mask_noarea.sum()
df = df[~mask_noarea]
print(f"Removed {num_noarea} rows with missing area.")

# 2.5. Replace Greenland (GRL) with Denmark (DNK)
df.loc[df['area'] == 'GRL', 'area'] = 'DNK'

# 3. Find rows where value == 0 and fix
mask_zero = df['value'] == 0
df.loc[mask_zero, 'value'] = 1 / (2 * df.loc[mask_zero, 'effective_sample_size'])
df.loc[mask_zero, 'standard_error'] = np.sqrt(
    df.loc[mask_zero, 'value'] * (1 - df.loc[mask_zero, 'value']) / df.loc[mask_zero, 'effective_sample_size']
)

# 4. Cast effective_sample_size to integer
df['effective_sample_size'] = df['effective_sample_size'].astype(int)

# 5. Drop rows where value > 1
mask_gt1 = df['value'] > 1
num_gt1 = mask_gt1.sum()
df = df[~mask_gt1]
print(f"Removed {num_gt1} rows where 'value' exceeded 1.")
print(f"Adjusted {mask_zero.sum()} rows where 'value' was zero.")

# 6. Save back to CSV
df.to_csv('input_data.csv', index=False)


Removed 0 rows with missing area.
Removed 0 rows where 'value' exceeded 1.
Adjusted 0 rows where 'value' was zero.
